In [9]:
from pyspark.sql import SparkSession
import csv
import pandas as pd

spark = SparkSession.builder.master("local").appName("lopputyo").getOrCreate()

df = spark.read.csv("merged19_20_21.csv", header=True, inferSchema=True)
df.show(vertical=True)

df.registerTempTable("example")


-RECORD 0----------------------------------
 id                 | 0                    
 aikatyyppi         | Vuosikertymä         
 kuukausi_nro       | 12                   
 vuosikuukausi      | 201912               
 vuosi              | 2019                 
 kunta_nro          | 20                   
 kunta_nimi         | Akaa                 
 ikaryhma           | 25-29                
 sukupuoli          | Mies                 
 etuus              | Työmarkkinatukiet... 
 korvausperuste     | Työllistymistä ed... 
 saaja_lkm          | 19                   
 korvattu_paiva_lkm | 3158                 
 maksettu_eur       | 120106.43            
-RECORD 1----------------------------------
 id                 | 1                    
 aikatyyppi         | Vuosikertymä         
 kuukausi_nro       | 12                   
 vuosikuukausi      | 201912               
 vuosi              | 2019                 
 kunta_nro          | 19                   
 kunta_nimi         | Aura      

c:\Lipasto\MassData23\mass_data_analysis_2023\venv\lib\site-packages\pyspark\sql\dataframe.py:330: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [3]:
df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- aikatyyppi: string (nullable = true)
 |-- kuukausi_nro: integer (nullable = true)
 |-- vuosikuukausi: integer (nullable = true)
 |-- vuosi: integer (nullable = true)
 |-- kunta_nro: integer (nullable = true)
 |-- kunta_nimi: string (nullable = true)
 |-- ikaryhma: string (nullable = true)
 |-- sukupuoli: string (nullable = true)
 |-- etuus: string (nullable = true)
 |-- korvausperuste: string (nullable = true)
 |-- saaja_lkm: integer (nullable = true)
 |-- korvattu_paiva_lkm: integer (nullable = true)
 |-- maksettu_eur: double (nullable = true)



In [4]:
df.groupBy('etuus').count().orderBy('count',ascending=False).show()

+--------------------+-------+
|               etuus|  count|
+--------------------+-------+
|            Yhteensä|1464095|
|Työmarkkinatukiet...|1373934|
|     Työmarkkinatuki|1373833|
|      Peruspäiväraha| 570001|
|      Kotoutumistuki|   2476|
+--------------------+-------+



In [21]:
new_df = df.withColumn("maksu_per_hlo", (df["maksettu_eur"]/df["saaja_lkm"])).orderBy('id',ascending=False).filter(df["aikatyyppi"] == "Kuukausi")
new_df = new_df.filter(new_df["maksettu_eur"] > 0)
new_df.show()


+-------+----------+------------+-------------+-----+---------+-----------+--------+---------+---------------+--------------------+---------+------------------+------------+------------------+
|     id|aikatyyppi|kuukausi_nro|vuosikuukausi|vuosi|kunta_nro| kunta_nimi|ikaryhma|sukupuoli|          etuus|      korvausperuste|saaja_lkm|korvattu_paiva_lkm|maksettu_eur|     maksu_per_hlo|
+-------+----------+------------+-------------+-----+---------+-----------+--------+---------+---------------+--------------------+---------+------------------+------------+------------------+
|4784338|  Kuukausi|          12|       202112| 2021|       78|      Hanko|   45-49|     Mies|Työmarkkinatuki|-Kuntouttava työt...|        7|               133|     5284.24| 754.8914285714285|
|4784337|  Kuukausi|          12|       202112| 2021|       91|   Helsinki|   45-49|     Mies|Työmarkkinatuki|-Kuntouttava työt...|       55|              1149|    46989.36|           854.352|
|4784336|  Kuukausi|          12|  

In [22]:
new_df.select("saaja_lkm", "maksettu_eur", "maksu_per_hlo").describe().show()

+-------+-----------------+------------------+------------------+
|summary|        saaja_lkm|      maksettu_eur|     maksu_per_hlo|
+-------+-----------------+------------------+------------------+
|  count|          1951238|           1951238|           1951238|
|   mean| 27.7847935515811|21328.101511089146| 760.2707943869406|
| stddev|94.14958040465211| 73781.94038589693|183.42200402020688|
|    min|                1|              0.08|              0.08|
|    max|             3760|         2785132.2|          16037.11|
+-------+-----------------+------------------+------------------+



In [18]:
new_df.select("kuukausi_nro", "maksu_per_hlo").groupBy('kuukausi_nro').avg().orderBy("kuukausi_nro").show()

+------------+-----------------+------------------+
|kuukausi_nro|avg(kuukausi_nro)|avg(maksu_per_hlo)|
+------------+-----------------+------------------+
|           1|              1.0|62.665051547937416|
|           2|              2.0|106.20113225335905|
|           3|              3.0|148.64316110509824|
|           4|              4.0|184.85781640967872|
|           5|              5.0|215.25298350679472|
|           6|              6.0|241.93374855860714|
|           7|              7.0| 267.2230089536965|
|           8|              8.0| 289.9562609628517|
|           9|              9.0| 308.1839519204293|
|          10|             10.0|327.25554941966595|
|          11|             11.0|345.52138592906863|
|          12|             12.0|367.02176958695816|
+------------+-----------------+------------------+



In [119]:
new_df.select("ikaryhma", "maksu_per_hlo").groupBy('ikaryhma').avg().orderBy("ikaryhma").show()

+-------------+------------------+
|     ikaryhma|avg(maksu_per_hlo)|
+-------------+------------------+
|        17-19| 667.8246949503166|
|        20-24| 714.8403877688473|
|        25-29| 761.7546401057336|
|        30-34| 794.9645386138585|
|        35-39| 817.3439506175276|
|        40-44| 812.6604219503253|
|        45-49| 787.8406128457792|
|        50-54|  752.552064522686|
|        55-59| 727.1541813808893|
|        60-64| 714.9705868728151|
|        65-67| 535.9984415595703|
|Tieto puuttuu| 619.9086500742875|
+-------------+------------------+



In [31]:
new_df.select("vuosi", "maksu_per_hlo").groupBy('vuosi').avg().orderBy("vuosi").show()

+-----+----------+------------------+
|vuosi|avg(vuosi)|avg(maksu_per_hlo)|
+-----+----------+------------------+
| 2019|    2019.0| 740.5814539456921|
| 2020|    2020.0| 767.1459090725434|
| 2021|    2021.0| 773.3091870413823|
+-----+----------+------------------+



In [121]:
new_df.select("Kunta_nimi", "maksu_per_hlo").groupBy("Kunta_nimi").avg().orderBy("avg(maksu_per_hlo)", ascending=False).show(300)


+------------------+------------------+
|        Kunta_nimi|avg(maksu_per_hlo)|
+------------------+------------------+
|         Ylitornio| 816.7491616413304|
|             Luoto|   812.91044608515|
|            Oripää| 810.6171082227651|
|        Kauniainen| 808.9986567222611|
|     Uusikaarlepyy| 803.8866197301277|
|         Siikainen| 803.0896097624169|
|          Utajärvi| 801.2340629758378|
|          Marttila| 796.5370195550943|
|       Punkalaidun| 793.4276284923425|
|            Kerava| 791.2382816333173|
|             Perho| 790.6351208876331|
|             Hanko| 785.7905625278909|
|            Raisio| 785.3242139856061|
|           Nakkila| 783.7887981172889|
|        Tuntematon| 783.6719292385851|
|  Pedersören kunta| 782.8731303450713|
|         Raasepori| 781.4985503427436|
|        Nurmijärvi| 781.3491708837942|
|       Pietarsaari| 780.9747688739747|
|             Inkoo| 780.8041376523386|
|          Pomarkku| 780.6214664157011|
|             Ypäjä| 779.4557838697059|


In [122]:
data_for_json = new_df.select("Kunta_nro", "maksu_per_hlo").groupBy("Kunta_nro").avg().orderBy("avg(Kunta_nro)", ascending=True)
data_for_json.show()


+---------+--------------+------------------+
|Kunta_nro|avg(Kunta_nro)|avg(maksu_per_hlo)|
+---------+--------------+------------------+
|        5|           5.0| 738.1062025981265|
|        9|           9.0| 727.9863972345393|
|       10|          10.0| 724.4748109763534|
|       16|          16.0| 738.7130065449091|
|       18|          18.0| 762.2738952728189|
|       19|          19.0| 754.4751288407152|
|       20|          20.0| 755.9141478496157|
|       46|          46.0| 751.8476794519348|
|       47|          47.0| 773.8661063921106|
|       49|          49.0| 621.0150824867189|
|       50|          50.0| 761.0217238217496|
|       51|          51.0| 769.7265981401238|
|       52|          52.0| 739.0193623854537|
|       61|          61.0| 775.3656286398626|
|       69|          69.0| 754.1616488032081|
|       71|          71.0| 711.5508237623047|
|       72|          72.0| 736.6703001983361|
|       74|          74.0| 753.0327456299973|
|       75|          75.0| 743.124

In [133]:
import json
with open("kunnat.json", "r") as kunnat:
    kuntajson = json.load(kunnat)
    

{'area005': '123', 'area009': 'Alavieska', 'area010': 'Alavus', 'area016': 'Asikkala', 'area018': 'Askola', 'area019': 'Aura', 'area020': 'Akaa', 'area035': 'BrÃ¤ndÃ¶', 'area043': 'EckerÃ¶', 'area046': 'Enonkoski', 'area047': 'EnontekiÃ¶', 'area049': 'Espoo', 'area050': 'Eura', 'area051': 'Eurajoki', 'area052': 'EvijÃ¤rvi', 'area060': 'FinstrÃ¶m', 'area061': 'Forssa', 'area062': 'FÃ¶glÃ¶', 'area065': 'Geta', 'area069': 'HaapajÃ¤rvi', 'area071': 'Haapavesi', 'area072': 'Hailuoto', 'area074': 'Halsua', 'area075': 'Hamina', 'area076': 'Hammarland', 'area077': 'Hankasalmi', 'area078': 'Hanko', 'area079': 'Harjavalta', 'area081': 'Hartola', 'area082': 'Hattula', 'area086': 'HausjÃ¤rvi', 'area090': 'HeinÃ¤vesi', 'area091': 'Helsinki', 'area092': 'Vantaa', 'area097': 'Hirvensalmi', 'area098': 'Hollola', 'area099': 'Honkajoki', 'area102': 'Huittinen', 'area103': 'Humppila', 'area105': 'Hyrynsalmi', 'area106': 'HyvinkÃ¤Ã¤', 'area108': 'HÃ¤meenkyrÃ¶', 'area109': 'HÃ¤meenlinna', 'area111': 'Heino

In [141]:



string = "{\n"
for row in data_for_json.rdd.collect():
    val1 = row["Kunta_nro"]
    val2 = row["avg(maksu_per_hlo)"]
    if f'area{val1:03d}' in kuntajson:
        string += f'"area{val1:03d}": "{val2:.2f}",\n'
string += "\n}"
print(string)


with open("kunnat2.json", "w") as kunnat2:
    json.dump(string, kunnat2)

{
"area005": "738.11",
"area009": "727.99",
"area010": "724.47",
"area016": "738.71",
"area018": "762.27",
"area019": "754.48",
"area020": "755.91",
"area046": "751.85",
"area047": "773.87",
"area049": "621.02",
"area050": "761.02",
"area051": "769.73",
"area052": "739.02",
"area061": "775.37",
"area069": "754.16",
"area071": "711.55",
"area072": "736.67",
"area074": "753.03",
"area075": "743.12",
"area077": "749.89",
"area078": "785.79",
"area079": "753.26",
"area081": "753.43",
"area082": "736.99",
"area086": "756.84",
"area090": "772.05",
"area091": "359.08",
"area092": "617.22",
"area097": "755.00",
"area098": "724.92",
"area099": "766.74",
"area102": "762.81",
"area103": "749.55",
"area105": "761.31",
"area106": "779.36",
"area108": "753.99",
"area109": "730.10",
"area111": "742.59",
"area139": "721.43",
"area140": "761.21",
"area142": "752.22",
"area143": "748.82",
"area145": "740.19",
"area146": "739.97",
"area148": "724.22",
"area149": "780.80",
"area151": "773.01",
"area152": 

In [144]:
jsondict = {}

for row in data_for_json.rdd.collect():
    val1 = row["Kunta_nro"]
    val2 = row["avg(maksu_per_hlo)"]
    #if f'area{val1:03d}' in kuntajson:
        #string += f'"area{val1:03d}": "{val2:.2f}",\n'
    jsondict[f'area{val1:03d}'] = f'{val2:.2f}'

with open("kunnat3.json", "w") as kunnat3:
    json.dump(jsondict, kunnat3)